In [ ]:
import cv2

# Load pre-trained classifiers for face, eyes, and smile detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Function to detect faces and estimate emotions
def detect_faces_and_estimate_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]  # Extract face region
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.2, minNeighbors=10)
        smiles = smile_cascade.detectMultiScale(face_roi, scaleFactor=1.8, minNeighbors=20)

        # Emotion estimation based on detected features
        if len(smiles) > 0 and len(eyes) > 0:
            emotion = "Happy 😀"
        elif len(smiles) == 0 and len(eyes) > 0 and h/w > 1.1:  # Face elongated (possible sadness)
            emotion = "Sad 😢"
        elif len(smiles) == 0 and len(eyes) > 1 and w/h > 1.1:  # Wide eyes, angry expression
            emotion = "Angry 😠"
        elif len(smiles) == 0 and len(eyes) > 1 and len(faces) > 1:  # Eyes wide, mouth open
            emotion = "Surprised 😲"
        else:
            emotion = "Neutral 😐"

        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Display estimated emotion
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return frame

# Open webcam
cap = cv2.VideoCapture(0)  # Use 0 for default webcam

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        break

    frame = detect_faces_and_estimate_emotion(frame)  # Detect faces and emotions
    cv2.imshow('Face Detection & Emotion Estimation', frame)  # Display frame

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()
